In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [89]:
from utils import rating

In [90]:
with open('../../data/processed/ratings.pkl', 'rb') as handle:
    ratings = pickle.load(handle)

In [91]:
gameweek_df = pd.read_pickle( "../../data/processed/gameweek_data.pkl")

In [92]:
raw_gameweek = pd.read_pickle('../../data/raw/gameweek_data.pkl')

In [93]:
player_df = pd.read_pickle('../../data/raw/player_data.pkl')

In [94]:
gameweek_df['home_team'] = np.where(gameweek_df.was_home, gameweek_df.team, gameweek_df.opponent_team)
gameweek_df['away_team'] = np.where(gameweek_df.was_home, gameweek_df.opponent_team, gameweek_df.team)

In [95]:
def result_map(home_score, away_score):
    if home_score > away_score:
        return 1.0
    elif home_score < away_score:
        return 0.0
    else:
        return 0.5

In [96]:
gameweek_df['result'] = gameweek_df.apply(lambda x: result_map(x.team_h_score, x.team_a_score),axis=1)

 find people who often have different team names from fixture with same score at same time (possible mid-season transfers)

In [97]:
gameweek_df['transfer_column'] =(gameweek_df.groupby(['kickoff_time','team_h_score','team_a_score',
             'home_team','away_team'])['team_h_score'].transform('count') < 11)

In [98]:
gameweek_df['transfer_likeliness'] = gameweek_df.groupby(['clean_name','season'])['transfer_column'].transform('mean')

only 50 or so players moving to different club in same window. 

In [99]:
(gameweek_df.loc[gameweek_df.transfer_likeliness > 0.05,'clean_name']).unique().shape[0]

55

If I exclude these guys, then I get pretty close to the 380 fixtures there should be per season

In [100]:
gameweek_df.loc[~((gameweek_df.transfer_likeliness > 0.05) &
                (gameweek_df.transfer_column< 11 )),['kickoff_time','team_h_score','team_a_score','home_team','away_team','season']].drop_duplicates().groupby('season')['team_h_score'].count()

season
2016-17    383
2017-18    383
2018-19    380
Name: team_h_score, dtype: int64

In [101]:
fixtures = gameweek_df.loc[~((gameweek_df.transfer_likeliness > 0.05) &
                (gameweek_df.transfer_column< 11 )),
                           ['kickoff_time','team_h_score','team_a_score',
                            'home_team','away_team','season','result']].drop_duplicates().sort_values(by='kickoff_time')

In [102]:
rated = pd.read_pickle('../../data/processed/matches_with_rating.pkl')

In [110]:
fixtures['Date']= pd.to_datetime(fixtures.kickoff_time).dt.tz_localize(None)
gameweek_df['Date']= pd.to_datetime(gameweek_df.kickoff_time).dt.tz_localize(None)


In [111]:
rated['FTHG'] = rated.FTHG.astype(int)
rated['FTAG'] =rated.FTAG.astype(int)

AttributeError: 'DataFrame' object has no attribute 'FTHG'

In [112]:
rated.rename(columns={'HomeTeam':'home_team',
                        'AwayTeam': 'away_team',
                        'FTHG':'team_h_score',
                     'FTAG':'team_a_score'},inplace=True)

In [113]:
test = pd.merge_asof(fixtures, rated, 
                     by=['home_team','away_team','team_h_score','team_a_score'], 
              on='Date', direction='nearest')

In [115]:
gameweek_with_elo = pd.merge_asof(gameweek_df.sort_values(by='Date'), rated, 
                     by=['home_team','away_team','team_h_score','team_a_score'], 
              on='Date', direction='nearest')

['elo_outcome_rating_1',
 'elo_outcome_rating_2',
 'elo_outcome_rating_diff',
 'elo_outcome_e']

In [122]:
gameweek_with_elo['elo_outcome_rating_p'] = np.where(gameweek_with_elo.was_home,
                                                    gameweek_with_elo.elo_outcome_rating_1,
                                                    gameweek_with_elo.elo_outcome_rating_2)

gameweek_with_elo['elo_outcome_rating_o'] = np.where(gameweek_with_elo.was_home,
                                                    gameweek_with_elo.elo_outcome_rating_2,
                                                    gameweek_with_elo.elo_outcome_rating_1)

gameweek_with_elo['elo_outcome_rating_d'] = np.where(gameweek_with_elo.was_home,
                                                    gameweek_with_elo.elo_outcome_rating_diff,
                                                    gameweek_with_elo.elo_outcome_rating_diff * -1)

gameweek_with_elo['elo_outcome_rating_ex'] = np.where(gameweek_with_elo.was_home,
                                                    gameweek_with_elo.elo_outcome_e,
                                                    1.0 - gameweek_with_elo.elo_outcome_e)


In [123]:
gameweek_with_elo.to_pickle('../../data/gameweek_with_elo.pkl')

In [86]:
rated.loc[
          (rated.Date > '2016'),'home_team'].value_counts()

Bradford                   13
Wigan                      12
Fleetwood Town             12
Oldham                     12
Bury                       12
Rochdale                   12
Blackpool                  12
Blackburn                  12
Swindon                    11
Sheffield Weds             11
Sheffield United           11
Reading                    11
Port Vale                  11
Doncaster                  11
Charlton                   11
Millwall                   11
Wolverhampton Wanderers    11
Middlesbrough              11
Bristol City               11
Burnley                    11
Huddersfield Town          11
Brentford                  11
Colchester                 11
Scunthorpe                 11
Nott'm Forest              11
Hull City                  11
Derby                      11
Crewe                      11
Ipswich                    11
Preston                    11
                           ..
Gillingham                 11
Barnsley                   11
Rotherham 

In [73]:
test.season.value_counts()

2016-17    102
2017-18     86
2018-19     66
Name: season, dtype: int64

In [ ]:
test.home_team.notna().mean()

In [ ]:
test.shape

In [ ]:
gameweek_df[['kickoff_time','team_h_score','team_a_score','home_team','away_team','season']].drop_duplicates().groupby('season')['team_h_score'].count()

In [ ]:
(gameweek_df
 [['kickoff_time','away_team',
   'team_h_score','team_a_score','season','result']]
.sort_values(by='kickoff_time')).groupby(['kickoff_time','team_a_score'])['result'].count()